In [116]:
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tabulate import tabulate
import pandas as pd
from great_tables import GT, loc, style

In [117]:
file = np.load("mkb_data.npz")
m_limits = [0, 10]; k_limits = m_limits; b_limits = m_limits

In [118]:
t_values = file["t"]
y_values = file["y"]

In [119]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x = t_values,
        y = y_values,
    )
)

fig.update_layout(
    template = "plotly_dark",
    xaxis = dict(
        title = "Time (s)"
    ),
    yaxis = dict(
        title = "Position (mts)"
    ),
    title = "Mass-Spring-Damper System Response"
)
fig.show()

## Mass-Spring-Damper formula

$$m \ddot{x} + b \dot{x} + kx = 5$$

Despejando para la acelaración:

$$\ddot{x} = - \frac{b \dot{x} + kx - 5}{m}$$

Se emplea el método de Euler para el cálculo de la velocidad y la posición del resorte

$$y(x_{i}+ h) = y(x_{i}) + y'(x_{i})h + \underbrace{R_{1}(x)}_{\leq \frac{h^{2}}{2} f'' ( \xi)}$$

In [120]:
class GeneticAlgorithm:

    def __init__(self, population_size, mutation_rate, generations):
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.generations = generations
        self.population = self.initialize_population()
        self.force = 5

    def initialize_population(self):
        population = []

        for _ in range(self.population_size):
            m = np.random.uniform(m_limits[0], m_limits[1])
            k = np.random.uniform(k_limits[0], k_limits[1])
            b = np.random.uniform(b_limits[0], b_limits[1])
            population.append((m, k, b))

        return population
    
    def fitness(self, individual):

        m, k, b = individual
        dt = t_values[1] - t_values[0]
        
        position = y_values[0]
        velocity = (y_values[1] - y_values[0]) / dt

        y_simulated = []

        for _ in range(len(t_values)):
            acceleration = -(b*velocity + k*position - self.force) / m
            velocity += acceleration * dt
            position += velocity * dt
            y_simulated.append(position)

        y_simulated = np.array(y_simulated)
        error = np.mean(np.abs(y_simulated - y_values))

        return -error


    def selection(self):

        fitness_scores = [self.fitness(ind) for ind in self.population]
        sort_indices = np.argsort(fitness_scores)[::-1]
        best_index = sort_indices[:2]

        return (self.population[best_index[0]], self.population[best_index[1]])


    def crossover(self, parent1, parent2):

        crossover_point = np.random.randint(1, 3)

        child1 = parent1[:crossover_point] + parent2[crossover_point:]
        child2 = parent2[:crossover_point] + parent1[crossover_point:]

        return child1, child2
    
    def mutate(self, individual):

        if np.random.rand() < self.mutation_rate:

            gene_index = np.random.randint(3)

            if gene_index == 0:

                individual = (
                    np.random.uniform(m_limits[0], m_limits[1]),
                    individual[1],
                    individual[2]
                )

            elif gene_index == 1:

                individual = (
                    individual[0],
                    np.random.uniform(k_limits[0], k_limits[1]),
                    individual[2]
                )

            else:

                individual = (
                
                    individual[0],
                    individual[1],
                    np.random.uniform(b_limits[0], b_limits[1])

                )

        return individual
    
    def run(self):

        best_overall = []
        best_individual = None
        best_fitness = float('-inf')

        for generation in range(self.generations):

            new_population = []

            for _ in range(self.population_size // 2):

                parent1, parent2 = self.selection()
                child1, child2 = self.crossover(parent1, parent2)

                child1 = self.mutate(child1)
                child2 = self.mutate(child2)

                new_population.extend([child1, child2])

            self.population = new_population

            for individual in self.population:

                fit = self.fitness(individual)

                if fit > best_fitness:

                    best_fitness = fit
                    best_individual = individual
                    best_overall.append((best_individual, best_fitness))

            print(f"Generation {generation+1}: Best Fitness = {best_fitness:.2f}, Best Individual = {best_individual}")

        return best_overall

In [121]:
MAX_GENERATIONS = 50
POPULATION_SIZE = 100
MUTATION_RATE = 0.3

In [122]:
ga = GeneticAlgorithm(POPULATION_SIZE, MUTATION_RATE, MAX_GENERATIONS)
best_overall = ga.run()

Generation 1: Best Fitness = -0.04, Best Individual = (2.5246718733006803, 8.562541711660419, 7.403189622718546)
Generation 2: Best Fitness = -0.03, Best Individual = (2.5246718733006803, 8.562541711660419, 3.498200312948759)
Generation 3: Best Fitness = -0.03, Best Individual = (1.566494826585163, 8.562541711660419, 3.498200312948759)
Generation 4: Best Fitness = -0.02, Best Individual = (1.566494826585163, 8.562541711660419, 1.3978771823191893)
Generation 5: Best Fitness = -0.01, Best Individual = (1.566494826585163, 8.562541711660419, 1.9503757039757086)
Generation 6: Best Fitness = -0.01, Best Individual = (1.566494826585163, 8.562541711660419, 1.9503757039757086)
Generation 7: Best Fitness = -0.01, Best Individual = (1.566494826585163, 8.562541711660419, 1.9503757039757086)
Generation 8: Best Fitness = -0.01, Best Individual = (1.566494826585163, 8.562541711660419, 1.9503757039757086)
Generation 9: Best Fitness = -0.01, Best Individual = (1.566494826585163, 8.562541711660419, 1.67

## Error table of the last 10 points

In [123]:
best_df = pd.DataFrame(
    [(m,k,b,fitness) for (m,k,b), fitness in best_overall],
    columns = ["m", "k", "b", "fitness_value"]
)

In [124]:
last_10 = best_df.tail(10)

In [132]:
tabla = (
    GT(last_10)
    .tab_header(
        title = "Diez últimos puntos del GA",
    )
    .cols_label(k = "k", b = "b", fitness_value = "Fitness")
    .fmt_number(columns = list(last_10.columns), decimals = 4)

    .tab_style(style = style.fill(color = "aliceblue"),  locations = loc.body(columns = ["m", "k", "b"]))
    .tab_style(style = style.fill(color = "papayawhip"), locations=loc.body(columns = "fitness_value"))

)

tabla


GT(_tbl_data=           m         k         b  fitness_value
11  1.566495  8.562542  1.672020      -0.008185
12  1.591069  8.562542  1.672020      -0.007112
13  1.591069  8.623512  1.672020      -0.007001
14  1.591069  8.568165  1.672020      -0.006898
15  1.591069  8.587725  1.672020      -0.006235
16  1.614307  8.623512  1.672020      -0.006127
17  1.614307  8.587725  1.672020      -0.005484
18  1.614307  8.603622  1.672020      -0.005321
19  1.614307  8.603622  1.809642      -0.005113
20  1.614307  8.603622  1.711048      -0.004561, _body=<great_tables._gt_data.Body object at 0x12e693ac0>, _boxhead=Boxhead([ColInfo(var='m', type=<ColInfoTypeEnum.default: 1>, column_label='m', column_align='right', column_width=None), ColInfo(var='k', type=<ColInfoTypeEnum.default: 1>, column_label='k', column_align='right', column_width=None), ColInfo(var='b', type=<ColInfoTypeEnum.default: 1>, column_label='b', column_align='right', column_width=None), ColInfo(var='fitness_value', type=<ColInfoTypeEnum.default: 1>, column_label='Fitness', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x12e5b1550>, _spanners=Spanners([]), _heading=Heading(title='Diez últimos puntos del GA', subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns=['m', 'k', 'b'], rows=None, mask=None), grpname=None, colname='m', rownum=0, colnum=None, styles=[CellStyleFill(color='aliceblue')]), StyleInfo(locname=LocBody(columns=['m', 'k', 'b'], rows=None, mask=None), grpname=None, colname='m', rownum=1, colnum=None, styles=[CellStyleFill(color='aliceblue')]), StyleInfo(locname=LocBody(columns=['m', 'k', 'b'], rows=None, mask=None), grpname=None, colname='m', rownum=2, colnum=None, styles=[CellStyleFill(color='aliceblue')]), StyleInfo(locname=LocBody(columns=['m', 'k', 'b'], rows=None, mask=None), grpname=None, colname='m', rownum=3, colnum=None, styles=[CellStyleFill(color='aliceblue')]), StyleInfo(locname=LocBody(columns=['m', 'k', 'b'], rows=None, mask=None), grpname=None, colname='m', rownum=4, colnum=None, styles=[CellStyleFill(color='aliceblue')]), StyleInfo(locname=LocBody(columns=['m', 'k', 'b'], rows=None, mask=None), grpname=None, colname='m', rownum=5, colnum=None, styles=[CellStyleFill(color='aliceblue')]), StyleInfo(locname=LocBody(columns=['m', 'k', 'b'], rows=None, mask=None), grpname=None, colname='m', rownum=6, colnum=None, styles=[CellStyleFill(color='aliceblue')]), StyleInfo(locname=LocBody(columns=['m', 'k', 'b'], rows=None, mask=None), grpname=None, colname='m', rownum=7, colnum=None, styles=[CellStyleFill(color='aliceblue')]), StyleInfo(locname=LocBody(columns=['m', 'k', 'b'], rows=None, mask=None), grpname=None, colname='m', rownum=8, colnum=None, styles=[CellStyleFill(color='aliceblue')]), StyleInfo(locname=LocBody(columns=['m', 'k', 'b'], rows=None, mask=None), grpname=None, colname='m', rownum=9, colnum=None, styles=[CellStyleFill(color='aliceblue')]), StyleInfo(locname=LocBody(columns=['m', 'k', 'b'], rows=None, mask=None), grpname=None, colname='k', rownum=0, colnum=None, styles=[CellStyleFill(color='aliceblue')]), StyleInfo(locname=LocBody(columns=['m', 'k', 'b'], rows=None, mask=None), grpname=None, colname='k', rownum=1, colnum=None, styles=[CellStyleFill(color='aliceblue')]), StyleInfo(locname=LocBody(columns=['m', 'k', 'b'], rows=None, mask=None), grpname=None, colname='k', rownum=2, colnum=None, styles=[CellStyleFill(color='aliceblue')]), StyleInfo(locname=LocBody(columns=['m', 'k', 'b'], rows=None, mask=None), grpname=None, colname='k', rownum=3, colnum=None, styles=[CellStyleFill(color='aliceblue')]), StyleInfo(locname=LocBody(columns=['m', 'k', 'b'], rows=None, mask=None), grpname=None, colname='k', rownum=4, colnum=None, styles=[CellStyleFill(color='aliceblue')]), StyleInfo(locname=LocBody(columns=['m', 'k', 'b'], rows=None, mask=None), grpname=None, colname='k', rownum=5, colnum=None, styles=[CellStyleFill(color='aliceblue')]), S

In [126]:
best_overall.sort(key = lambda value: value[1], reverse = True)
best_m, best_k, best_b = best_overall[0][0]

In [127]:
print(f"Best values of m = {best_m:.4f}, k = {best_k:.4f}, b = {best_b:.4f}")

Best values of m = 1.6143, k = 8.6036, b = 1.7110


## Graphs of the points and the final identified system

In [128]:
dt = t_values[1] - t_values[0]

position = y_values[0]
velocity = (y_values[1] - y_values[0]) / dt

y_simulated = []
force = 5

for _ in range(len(t_values)):
    acceleration = -(best_b*velocity + best_k*position - force) / best_m
    velocity += acceleration * dt
    position += velocity * dt
    y_simulated.append(position)

y_simulated = np.array(y_simulated)
error = np.mean(np.abs(y_simulated - y_values))

In [129]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x = t_values,
        y = y_simulated,
        mode = "lines",
        name = "approximation"
    )
)

fig.add_trace(
    go.Scatter(
        x = t_values,
        y = y_values,
        mode = "lines",
        name = "real values"
    )
)

fig.update_layout(
    template = "plotly_dark",
    xaxis = dict(
        title = "Time (s)"
    ),
    yaxis = dict(
        title = "Position (mts)"
    ),
    title = "Simulation vs Real Values"
)


fig.show()